# Document Loader and Q&A
This script will load all PDFs from a directory and then upload to a vector store

PyPDFDirectoryLoader loads all PDFs in a folder one page at a time (e.g. a PDF with 6 pages will produce a loader with a length of six).

In [20]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyMuPDFLoader
import os
from dotenv import load_dotenv

OPENAI_API_KEY = load_dotenv("OPENAI_API_KEY")

target_directory = "/home/stubbs/Documents/pdfs/"

for pdf_file in os.listdir(target_directory):
    print(pdf_file)
    
    loader = PyMuPDFLoader(pdf_file)
    data = loader.load()
    print(data[0])

# loader = PyPDFDirectoryLoader("/home/stubbs/Documents/pdfs")
# docs = loader.load()

accepts-dbt.pdf


ImportError: `PyMuPDF` package not found, please install it with `pip install pymupdf`

In [8]:
print(f'You have {len(docs)} pages in your PDFs')
print(f'There are {len(docs[0].page_content)} ')

You have 10 pages in your PDFs
There are 280 


In [9]:
print(docs)

[Document(page_content='DBT Skills Training Handouts and Worksheets, Second Edition\nDBT Skills Training Handouts and Worksheets, Second Edition DBT Skills \nTraining Manual, Second EditionDistracting\n“Wise Mind ACCEPTS.”\nActivities:\nContributing:\nComparisons:\nEmotions:\nPushing away:\nThoughts:\nSensations:', metadata={'source': '/home/stubbs/Documents/pdfs/accepts-dbt.pdf', 'page': 0}), Document(page_content=' “STOP” Skill  \n  \n   Find more online:  DBT.tools      1 of 2 The STOP  skill consists of the following sequence: Stop, Take a step back, Observe, Proceed.  \n \nStop \nWhen you feel that your emotions seem to be in control, stop! Don’t react. Don’t move a muscle. Just freeze \n(every muscle). Freezing for a moment helps prevent you from doing what your emotions want you to do \n(which is to act without thinking). Stay in control. Remember, you are the boss of your emotions. Name the \nemotion – put a label on it.  \nTake a step back  \nWhen you are faced with a difficu 